# Preprocessing

## Generating crack patches

Generate crack patches size of 64

In [1]:
import os
import numpy as np
import sys
import json
import datetime
import skimage.draw
import cv2
import math
import tensorflow as tf
import scipy
import skimage.color
import skimage.io
import skimage.transform
from matplotlib import pyplot as plt

C:\Users\kwon\anaconda3\envs\GPUtest2\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kwon\anaconda3\envs\GPUtest2\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kwon\anaconda3\envs\GPUtest2\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kwon\anaconda3\envs\GPUtest2\lib\site-packages\tensorflow\

In [2]:
def preprocessing(dataset_dir, subset, patch_size=64):
    
    assert subset in ["train", "val"]
    dataset_dir = os.path.join(dataset_dir, subset)
    
    annotations = json.load(open(os.path.join(dataset_dir,"label.json")))
    for a in annotations: # each images
        label = a['Label']['objects']
        print(a['External ID'])
        image_path = os.path.join(dataset_dir,a['External ID'])
        image = skimage.io.imread(image_path)
        # print(label)
        oh, ow = image.shape[:2]
        print("Original height, width", oh, ow)
        
        crack_num = 0
        for r in label: # each crack
            if r['title'] == 'crack' and r['line']:
                track = 0
                for k in r['line']: # each point
                    flag = True
                    p = [0, 0]
                    if track == 0 or track == (len(r['line']) - 1): p = [k['x'], k['y']]
                    else:
                        p1 = np.array([r['line'][track-1]['x'], r['line'][track-1]['y']])
                        p2 = np.array([r['line'][track+1]['x'], r['line'][track+1]['y']])
                        dis1, dis2 = np.linalg.norm(p1-np.array([k['x'], k['y']])), np.linalg.norm(p2-np.array([k['x'], k['y']]))
                        if dis1 >= 64 and dis2 >= 64: p = [k['x'], k['y']]
                        else: flag = False

                    if flag:
                        # Check the coordinate in the image
                        if ((p[0]+32<ow and p[0]-32>0) and (p[1]+32<oh and p[1]-32>0)):
                            new_image = image[int(p[1]-32):int(p[1]+32), int(p[0]-32):int(p[0]+32)]
                        else:
                            print(p)
                            # Case 1
                            if p[1] - 32 <= 0: 
                                if p[0] - 32 <= 0: new_image = image[:64, :64]
                                elif p[0] + 32 > ow: new_image = image[:64, int(ow-64):ow]
                                else: new_image = image[:64, int(p[0]-32):int(p[0]+32)]
                            elif p[1] + 32 >= oh:
                                if p[0] - 32 <= 0: new_image = image[oh-64:oh, :64]
                                elif p[0] + 32 > ow: new_image = image[oh-64:oh, ow-64:ow]
                                else: new_image = image[oh-64:oh, int(p[0]-32):int(p[0]+32)]
                            else:
                                if p[0] - 32 <= 0: new_image = image[int(p[1]-32):int(p[1]+32), :64]
                                elif p[0] + 32 > ow: new_image = image[int(p[1]-32):int(p[1]+32), ow-64:ow]
                                else: new_image = image[int(p[1]-32):int(p[1]+32), int(p[0]-32):int(p[0]+32)]

                        new_filename = a['External ID'].split('.')[0] + "_" + str(crack_num) + "_" + str(track) + ".jpg"
                        dest_dir = dataset_dir+"/Crack_temp/"+new_filename
                        cv2.imwrite(dest_dir, new_image)

                        # increment track number
                        track += 1
            
            #increment crack_num
            crack_num += 1


In [3]:
preprocessing(r'C:/Users/kwon/Mobile_Mask_RCNN/data/deficiency',r'val')

Calderone-2000-1028-14.58%-2.jpg
Original height, width 403 601
Correal-2007-ISH1.0-6.67%.jpg
Original height, width 609 489
CIP Test 045.jpg
Original height, width 3648 2048
[3.526, 1517.172]
[2025.964, 1524.223]
[2043.964, 1508.889]
[2053.964, 1452.889]
[2062.631, 1431.223]
[2046.964, 1412.39]
[2055.131, 1398.307]
[2053.964, 1386.89]
[2096.797, 420.056]
[2123.464, 432.723]
[2128.131, 441.389]
[2239.123, 320.06]
[2229.123, 480.06]
[2235.123, 536.06]
[2249.123, 582.06]
[2237.123, 612.06]
[2223.123, 648.06]
[2215.123, 692.06]
[2217.123, 728.06]
[2215.123, 776.06]
[2201.123, 812.06]
[2181.123, 852.06]
[2151.123, 860.06]
[2123.123, 900.06]
[2105.123, 928.06]
[2087.123, 966.06]
[2085.123, 1008.06]
[2087.123, 1046.06]
[2069.123, 1074.06]
[2083.123, 1108.06]
[2386.123, 120.727]
[2378.123, 215.727]
[2369.123, 228.727]
[2372.123, 245.227]
[2387.123, 257.727]
[2400.123, 283.727]
[2401.123, 308.727]
[2394.123, 338.727]
[2396.123, 367.727]
[2395.123, 420.727]
[2398.123, 453.727]
[2400.123, 480.72

In [4]:
preprocessing(r'C:/Users/kwon/Mobile_Mask_RCNN/data/deficiency',r'train')

CIP Test 039.jpg
Original height, width 3648 2048
[1176.376, 26.535]
[1194.043, 20.868]
[1211.376, 20.868]
[1217.209, 19.702]
[1236.418, 18.827]
[1248.376, 13.535]
[1264.043, 11.035]
[1282.043, 20.868]
[1296.543, 27.368]
[1304.793, 29.285]
[1315.376, 31.202]
[1327.876, 24.868]
[1334.459, 16.702]
[1344.376, 8.535]
[1366.709, 1.202]
[1379.209, 7.868]
[1388.376, 10.202]
[1401.709, 10.369]
[1422.709, 15.785]
[1434.876, 16.493]
[1444.376, 19.868]
[1454.709, 31.868]
[1467.543, 31.201]
[1420.089, 0.575]
[1504.006, 17.075]
[1520.173, 19.075]
[1524.756, 23.575]
[1532.756, 31.242]
CIP Test 071.jpg
Original height, width 3648 2048
CIP Test 033.jpg
Original height, width 3648 2048
[1011.318, 29.646]
[1081.198, 17.597]
CIP Test 065.jpg
Original height, width 3648 2048
[857.137, 21.996]
[885.137, 5.996]
[1177.104, 13.214]
[1233.104, 5.214]
CIP Test 097.jpg
Original height, width 3648 2048
[738.258, 9.197]
[778.258, 13.197]
[812.258, 7.197]
[992.612, 18.49]
[1049.653, 6.95]
[1107.089, 15.638]
[1126.1

Original height, width 428 508
[190.838, 421.816]
Phan-2005-NF_1-13.3%.jpg
Original height, width 570 438
Wu-2017-F_CIP_O-8.02%_2.jpg
Original height, width 463 449
[423.456, 181.061]
[134.706, 7.394]
[353.206, 7.894]
[399.206, 6.644]
Al-Jelawy-2018-C_40_1-5%.jpg
Original height, width 555 413
[101.113, 5.728]
[268.951, 4.936]
[316.201, 13.186]
Chan-2020-CIP25-6%.jpg
Original height, width 533 277
[75.311, 10.704]
[21.978, 65.454]
[25.061, 98.12]
Correal-2007-ISH1.5-1.87%-2.jpg
Original height, width 483 367
[193.197, 12.18]
[206.197, 31.43]
[229.947, 28.68]
[227.697, 31.18]
[271.281, 20.764]
[137.281, 7.014]
[282.197, 31.597]


In [18]:
arr1 = np.array([3, 2])
arr2 = np.array([7, 1])

np.linalg.norm(arr1-arr2)

4.123105625617661